In [ ]:
import sys; sys.path.append('..')
import numpy as np, elastic_rods, time
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer
from matplotlib import pyplot as plt
import os
from open_linkage import open_linkage

In [ ]:
def resultsDirForRun(inputName, aspectRatio):
    return f'results/{inputName}/aspect_ratio_{aspectRatio}'

## Run the simulations/analyses for all meshes

In [ ]:
def runAnalysis(inputName, aspectRatio):
    lineMesh = inputName + '.obj'
    l = elastic_rods.RodLinkage(lineMesh, 10)
    driver=l.centralJoint()

    width = 0.01
    thickness = width / aspectRatio
    resultDir = resultsDirForRun(inputName, aspectRatio)
    os.makedirs(resultDir, exist_ok=True)

    l.setMaterial(elastic_rods.RodMaterial('rectangle', 20000, 0.3, [.01, thickness]))
    l.setBendingEnergyType(elastic_rods.BendingEnergyType.Bergou2008)

    l.saveVisualizationGeometry(f'{resultDir}/pre_restlen_solve.msh')

    opts = elastic_rods.NewtonOptimizerOptions()
    opts.niter = 5000

    with suppress_stdout(): elastic_rods.restlen_solve(l, opts)

    l.saveVisualizationGeometry(f'{resultDir}/post_restlen_solve.msh')

    jdo = l.dofOffsetForJoint(driver)
    fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
    
    opts = elastic_rods.NewtonOptimizerOptions()
    opts.niter = 5
    opts.beta = 1e-12
    opts.gradTol = 1e-12

    equilibriumFramesDir = f'{resultDir}/equilibriumFrames'
    os.makedirs(equilibriumFramesDir, exist_ok=True)

    for i in range(1000):
        with suppress_stdout(): cr = elastic_rods.compute_equilibrium(l, fixedVars=fixedVars, options=opts)
        l.saveVisualizationGeometry(f'{equilibriumFramesDir}/step_{i}.msh')
        if (cr.success or (cr.numIters() == 1)): break
            
    lambdas, mview = None, None
    if (aspectRatio != 20): # For some reason aspect ratio 20 is taking forever...
        import compute_vibrational_modes
        lambdas, modes = compute_vibrational_modes.compute_vibrational_modes(l, fixedVars=[], mtype=compute_vibrational_modes.MassMatrixType.FULL, n=9)
        modes = modes[:, 6:]
        lambdas = lambdas[6:]

        import mode_viewer, importlib
        importlib.reload(mode_viewer);
        mview = mode_viewer.ModeViewer(l, modes, lambdas, amplitude=5.0, width=768, height=640)
    
    import deployment_path_analysis
    dpa = deployment_path_analysis.deploymentPathAnalysis(l, fixedVars)
    deployment_mview = deployment_path_analysis.deploymentModeViewer(l, fixedVars)
    deployment_mview.setAmplitude(0.3)
    
    def MNormSq(v): return l.massMatrix().apply(v).dot(v)
    def EnergyIncrement(v): return l.hessian().apply(v).dot(v)
    d = dpa.deploymentStep / np.sqrt(MNormSq(dpa.deploymentStep))
    
    def equilibriumSolver(tgtAngle, l, opts, fv):
        opts.beta = 1e-8
        opts.gradTol = 1e-12
        opts.useIdentityMetric = False
        opts.niter = 1000
        return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)
    # We copy the linkage so the linkage referenced by mview/deployment_mview is left intact
    lclose = elastic_rods.RodLinkage(l)
    closeFramesDir = f'{resultDir}/closeFrames'
    os.makedirs(closeFramesDir, exist_ok=True)
    with suppress_stdout(): cr = open_linkage(lclose, driver, np.deg2rad(50) - l.averageJointAngle, 100, None, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=80, verbose=10, useTargetAngleConstraint=True,
                                              outPathFormat=closeFramesDir + '/frame_{}.msh')

    energies = [r.energy[-1] for r in cr[0]][:-1]
    angles = cr[2][:-1]
    bend_energies = [e['energy_bend'] for e in [r.customData[-1] for r in cr[0]][:-1]]
    twist_energies = [e['energy_twist'] for e in [r.customData[-1] for r in cr[0]][:-1]]
    fig = plt.figure(figsize=(10,6))
    plt.plot(angles, energies, angles, bend_energies, angles, twist_energies)
    plt.title('Energy along path')
    plt.xlabel('Average opening angle')
    plt.legend(['Total', 'Bend', 'Twist'])
    plt.tight_layout()
    plt.grid()
    plt.savefig(resultDir + '/close_path_energy.png', dpi=144)
    
    return (mview, deployment_mview, lambdas, EnergyIncrement(d))

In [ ]:
results = {}
for inputName in ['scherk_1_good_x_cleaned', 'scherk_1_good_relaxed_x_cleaned']:
    for aspectRatio in [5, 10, 20]:
        print(inputName, aspectRatio)
        results[(inputName, aspectRatio)] = runAnalysis(inputName, aspectRatio)

## Generate movies for the equilibrium solves and closing simulations

In [ ]:
import os
from glob import glob
import subprocess
import PIL
from PIL import Image, ImageChops

def imageForMesh(path):
    num = path.split('_')[-1].split('.')[0]
    return f'{os.path.dirname(path)}/frame_{num}.png'
    
def meshesForRun(inputName, aspectRatio):
    resultsDir = resultsDirForRun(inputName, aspectRatio)
    # return glob(f'{resultsDir}/equilibriumFrames/*.msh')
    return [path for subdir in ['equilibriumFrames', 'closeFrames'] for path in glob(f'{resultsDir}/{subdir}/*.msh')]

def renderMesh(mesh, outImage):
    subprocess.call(['gmsh_offscreen', '-n', 'results/render3.opt', mesh, '-o', outImage])

def renderImages(inputName, aspectRatio):
    for mesh in meshesForRun(inputName, aspectRatio):
        outImage = imageForMesh(mesh)
        renderMesh(mesh, imageForMesh(mesh))
        
def processImages(inputName, aspectRatio):
    imgs = []
    pngPaths = []
    common_bbox = [np.inf, np.inf, -np.inf, -np.inf]
    # Resize and determine common clipping box
    for mesh in meshesForRun(inputName, aspectRatio):
        pngPath = imageForMesh(mesh)
        img = Image.open(pngPath)
        resized = img.resize([s // 2 for s in img.size], resample=PIL.Image.LANCZOS)
        imgs.append(resized)
        pngPaths.append(pngPath)
        bbox = ImageChops.invert(resized).getbbox()
        common_bbox[0] = min(common_bbox[0], bbox[0])
        common_bbox[1] = min(common_bbox[1], bbox[1])
        common_bbox[2] = max(common_bbox[2], bbox[2])
        common_bbox[3] = max(common_bbox[3], bbox[3])
        # print(common_bbox)
    # Expand bbox to nearest even coordinates
    common_bbox[0] = 2 * ( common_bbox[0]      // 2)
    common_bbox[1] = 2 * ( common_bbox[1]      // 2)
    common_bbox[2] = 2 * ((common_bbox[2] + 1) // 2)
    common_bbox[3] = 2 * ((common_bbox[3] + 1) // 2)
    # Crop and save
    for img, path in zip(imgs, pngPaths):
        img.crop(common_bbox).save(path + '.processed.png')

def genMovies(inputName, aspectRatio):
    resultsDir = resultsDirForRun(inputName, aspectRatio)
    for movie in ['equilibrium', 'close']:
        subprocess.call(['ffmpeg', '-y', '-f', 'image2', '-framerate', '24',
                         '-i', f'{resultsDir}/{movie}Frames/frame_%d.png.processed.png',
                         '-c:v', 'libx264',
                         '-preset', 'veryslow',
                         '-qp', '18',
                         '-pix_fmt', 'yuv420p',
                         f'{resultsDir}/{movie}.mp4'])

## Also render the pre/post restlength solves (keep in higher resolution) + convert mesh

In [ ]:
def rreplace(s, old, new, count = 1):
    li = s.rsplit(old, count)
    return new.join(li)

def renderAndConvertInitialMeshes(inputName, aspectRatio):
    resultsDir = resultsDirForRun(inputName, aspectRatio)
    for i in ['pre', 'post']:
        mshPath = f'{resultsDir}/{i}_restlen_solve.msh'
        imgPath = f'{resultsDir}/{i}_restlen_solve.png'
        renderMesh(mshPath, imgPath)
        img = Image.open(imgPath)
        img.crop(ImageChops.invert(img).getbbox()).save(imgPath)
        subprocess.call(['mesh_convert', mshPath, rreplace(mshPath, 'msh', 'obj')])

### Do it in parallel...

In [ ]:
from multiprocessing import Pool
def processAspectRatio(aspect):
    for structure in ['scherk_1_good_relaxed_x_cleaned', 'scherk_1_good_x_cleaned']:
        renderImages (structure, aspect)
        processImages(structure, aspect)
        genMovies    (structure, aspect)
        renderAndConvertInitialMeshes(structure, aspect)
        
with Pool(3) as p:
    p.map(processAspectRatio, [5, 10, 20])

## Generate html reports

In [ ]:
def htmlReportString(structure, aspect_ratio):
    mview, deployment_mview, lambdas, dstiff = results[(structure, aspect_ratio)]
    modeString = ''
    if (aspect_ratio != 20):
        lambda0 = lambdas[0] 
        lambda1 = lambdas[1] 
        modeString=f'''
    Lowest energy modes:<br>
        <a href='../../mode_viewers/{structure}_{aspect_ratio}_0.html'>mode 0 (stiffness = {lambda0:0.4f}MPa)</a><br>
        <a href='../../mode_viewers/{structure}_{aspect_ratio}_1.html'>mode 1 (stiffness = {lambda1:0.4f}MPa)</a><br><br>
        '''
        
    return f'''<html>
<head>
    <title>{structure}, ribbon aspect ratio {aspect_ratio}</title>
</head>
<body>
    <h2>{structure}, ribbon aspect ratio {aspect_ratio}</h2>
    <table>
        <tr>
            <td><img src="pre_restlen_solve.png" width=384> </td>
            <td><img src="post_restlen_solve.png" width=384></td>
        </tr>
        <tr>
            <td align='center'>Pre rest length solve (<a href='pre_restlen_solve.obj'>download obj</a>)</td>
            <td align='center'>Post rest length solve (<a href='post_restlen_solve.obj'>download obj</a>)</td>
        </tr>

        <tr>
            <td>
                <video controls width="512">
                    <source src="equilibrium.mp4" type="video/mp4">
                    Sorry, your browser doesn't support embedded videos.
                </video>
            </td>
            <td>
                <video controls width="512">
                    <source src="close.mp4" type="video/mp4">
                    Sorry, your browser doesn't support embedded videos.
                </video>
            </td>
        <tr>
            <td align="center">
                Equilibrium solve
            </td>
            <td align="center">
                Closing simulation (quasi-static)
            </td>
        </tr>
    </table>
    <br><br>

    <img src="close_path_energy.png" width=512>
    <br><br>
    {modeString}
    <a href='../../mode_viewers/{structure}_{aspect_ratio}_close.html'>Closing mode (stiffness = {dstiff:0.4f}MPa)</a><br><br>
</body>
</html>'''
def writeHTMLReport(structure, aspect_ratio):
    resultDir = resultsDirForRun(structure, aspect_ratio)
    print(htmlReportString(structure, aspect_ratio), file=open(f'{resultDir}/report.html', 'w'))

In [ ]:
for structure in ['scherk_1_good_relaxed_x_cleaned', 'scherk_1_good_x_cleaned']:
    for aspectRatio in [5, 10, 20]:
        writeHTMLReport(structure, aspectRatio)

## Manually output mode viewers.

In [ ]:
mview, deployment_mview, lambdas, dstiff = results[('scherk_1_good_relaxed_x_cleaned', 20)]

In [ ]:
mview.exportHTML('results/mode_viewers/scherk_1_good_x_cleaned_10_1.html')

In [ ]:
deployment_mview.exportHTML('results/mode_viewers/scherk_1_good_relaxed_x_cleaned_20_close.html')